In [59]:
import os
import json
import random
import pandas as pd
import shutil

In [60]:
!mkdir prepare_data/img_clf_multilabel_lst/train_imgs
!mkdir prepare_data/img_clf_multilabel_lst/valid_imgs
!mkdir prepare_data/img_clf_multilabel_lst/train_annots
!mkdir prepare_data/img_clf_multilabel_lst/valid_annots

In [61]:
gt_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/all_images_gt/clf_labels.csv", header=None
)
gt_df.columns = ["image", "class"]
gt_df.head()

,image,class
0,410.jpg,"[White,Female,Extension services,Middle,Square..."
1,544.jpg,"[White,Female,Extension services,Middle,Potrai..."
2,551.jpg,"[White,Female,Extension services,Middle,Potrai..."
3,Untitled design (21).jpg,"[White,Female,Extension services,Middle,Square..."
4,Untitled design (22).jpg,"[White,Female,Extension services,Middle,Square..."


In [62]:
train_valid_split = 0.7
nimages = gt_df["image"].nunique()
ntrain = int(train_valid_split * nimages)
nvalid = nimages - ntrain
print(nimages, ntrain, nvalid)

25 17 8


In [63]:
gt_df["class"] = gt_df["class"].apply(lambda x: x[1:-1])
gt_df.head()

,image,class
0,410.jpg,"White,Female,Extension services,Middle,Square,..."
1,544.jpg,"White,Female,Extension services,Middle,Potrait..."
2,551.jpg,"White,Female,Extension services,Middle,Potrait..."
3,Untitled design (21).jpg,"White,Female,Extension services,Middle,Square,..."
4,Untitled design (22).jpg,"White,Female,Extension services,Middle,Square,..."


In [64]:
labels =["anesthesia","Upper Body Wax","restorative dentistry","beauty training","Female","Kids","nail services","cosmetic injectables","mens wax","Square","mens colour","Extension services","Old","beauty treatment","portrait photography","grooming","business photography","wispy","Barber","plasma fibrolast","medical services","vitamin injections","wrinkles","microneedling","alternative therapy","wedding services","reflexology","teeth","tatto service","Hair Highlights","Right","body massage","chemical peels","group fitness classes","sugaring","physical therapy","lip treatments","Male","Bohemian Box Braids","piercing services","Locs","body waxing","Landscape","other","dreadlocs","passion twist","Left","cat eye","microblading","none","dermal fillers","sew ins","eyelash services","Gel Nails","lifestyle photography","weight loss services","waxing services","loc services","stretch mark treatment","group services","hair saloons","vajacial","acupunture","Hard Wax","perm","White","energy healing","fitness programs","Braids","Middle","Manicure","cornrows","crystal healing","detoxification services","crochet","eyebrow waxing","Eyebrows","Lower Body Wax","bridal services","Black","Body Contouring","Bridal","Strip Wax","updos","spa","grocery shopping","kids services","bikini wax","hair treatments","wig services","hair styling","artistic services","awakening programs","Dreads","feed ins","makeup services","Eyelash fills","nail polish","facial treatments","beauty services","weaves","Center","Knotless Braids","Brown","cupping therapy","microdermabrasion","personal training","eyebrow services","Pedicure","permanent makeup","skin care services","meditation","iv therepy","Potrait","Eyelash Lift","Acne treatment","yoga and meditation","tanning services","pilates classes","cosmetic dentistry","Box Braids","cooking classes","mens braids","discount services","bohemian twist","barber services","hard","Soft Wax","Facial waxing","light therepy","kinky twist"]
labels_map = {k: v for v, k in enumerate(labels)}
with open("prepare_data/img_clf_multilabel_lst/labels_map.json", "w") as fp:
    json.dump(labels_map, fp)
labels_map

{'anesthesia': 0,
 'Upper Body Wax': 1,
 'restorative dentistry': 2,
 'beauty training': 3,
 'Female': 4,
 'Kids': 5,
 'nail services': 6,
 'cosmetic injectables': 7,
 'mens wax': 8,
 'Square': 9,
 'mens colour': 10,
 'Extension services': 11,
 'Old': 12,
 'beauty treatment': 13,
 'portrait photography': 14,
 'grooming': 15,
 'business photography': 16,
 'wispy': 17,
 'Barber': 18,
 'plasma fibrolast': 19,
 'medical services': 20,
 'vitamin injections': 21,
 'wrinkles': 22,
 'microneedling': 23,
 'alternative therapy': 24,
 'wedding services': 25,
 'reflexology': 26,
 'teeth': 27,
 'tatto service': 28,
 'Hair Highlights': 29,
 'Right': 30,
 'body massage': 31,
 'chemical peels': 32,
 'group fitness classes': 33,
 'sugaring': 34,
 'physical therapy': 35,
 'lip treatments': 36,
 'Male': 37,
 'Bohemian Box Braids': 38,
 'piercing services': 39,
 'Locs': 40,
 'body waxing': 41,
 'Landscape': 42,
 'other': 43,
 'dreadlocs': 44,
 'passion twist': 45,
 'Left': 46,
 'cat eye': 47,
 'microbladi

In [65]:
def function(x):
    xl = x.split(",")
    xll = []
    for lbl in labels:
        if lbl in xl:
            xll.append(1)
        else:
            xll.append(0)
    return xll

In [66]:
gt_df["class2"] = gt_df["class"].apply(function)
gt_df.head()

,image,class,class2
0,410.jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
1,544.jpg,"White,Female,Extension services,Middle,Potrait...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
2,551.jpg,"White,Female,Extension services,Middle,Potrait...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
3,Untitled design (21).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
4,Untitled design (22).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."


In [67]:
gt_df[labels] = gt_df["class2"].apply(pd.Series)
gt_df.head()

/tmp/ipykernel_11897/2254870918.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gt_df[labels] = gt_df["class2"].apply(pd.Series)
/tmp/ipykernel_11897/2254870918.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gt_df[labels] = gt_df["class2"].apply(pd.Series)
/tmp/ipykernel_11897/2254870918.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-f

,image,class,class2,anesthesia,Upper Body Wax,restorative dentistry,beauty training,Female,Kids,nail services,...,cooking classes,mens braids,discount services,bohemian twist,barber services,hard,Soft Wax,Facial waxing,light therepy,kinky twist
0,410.jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,544.jpg,"White,Female,Extension services,Middle,Potrait...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,551.jpg,"White,Female,Extension services,Middle,Potrait...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Untitled design (21).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Untitled design (22).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
gt_df = gt_df.sample(frac=1).reset_index(drop=True)
gt_df.head()

,image,class,class2,anesthesia,Upper Body Wax,restorative dentistry,beauty training,Female,Kids,nail services,...,cooking classes,mens braids,discount services,bohemian twist,barber services,hard,Soft Wax,Facial waxing,light therepy,kinky twist
0,barer_10.jpg,"White,Male,Barber, barber services,Middle,Squa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ht_16.jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,extensionservicces_1 (13).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Untitled design (22).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,barer_13.jpg,"White,Male,Barber, barber services,Middle,Squa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
gt_df["index"] = gt_df.index + 1
gt_df.head()

,image,class,class2,anesthesia,Upper Body Wax,restorative dentistry,beauty training,Female,Kids,nail services,...,mens braids,discount services,bohemian twist,barber services,hard,Soft Wax,Facial waxing,light therepy,kinky twist,index
0,barer_10.jpg,"White,Male,Barber, barber services,Middle,Squa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ht_16.jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2,extensionservicces_1 (13).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3
3,Untitled design (22).jpg,"White,Female,Extension services,Middle,Square,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,4
4,barer_13.jpg,"White,Male,Barber, barber services,Middle,Squa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [70]:
sel_cols = ["index"] + labels + ["image"]
print(sel_cols)

['index', 'anesthesia', 'Upper Body Wax', 'restorative dentistry', 'beauty training', 'Female', 'Kids', 'nail services', 'cosmetic injectables', 'mens wax', 'Square', 'mens colour', 'Extension services', 'Old', 'beauty treatment', 'portrait photography', 'grooming', 'business photography', 'wispy', 'Barber', 'plasma fibrolast', 'medical services', 'vitamin injections', 'wrinkles', 'microneedling', 'alternative therapy', 'wedding services', 'reflexology', 'teeth', 'tatto service', 'Hair Highlights', 'Right', 'body massage', 'chemical peels', 'group fitness classes', 'sugaring', 'physical therapy', 'lip treatments', 'Male', 'Bohemian Box Braids', 'piercing services', 'Locs', 'body waxing', 'Landscape', 'other', 'dreadlocs', 'passion twist', 'Left', 'cat eye', 'microblading', 'none', 'dermal fillers', 'sew ins', 'eyelash services', 'Gel Nails', 'lifestyle photography', 'weight loss services', 'waxing services', 'loc services', 'stretch mark treatment', 'group services', 'hair saloons', 'v

In [71]:
gt_df[sel_cols].head(ntrain).to_csv(
    "prepare_data/img_clf_multilabel_lst/train_annots/train.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 prepare_data/img_clf_multilabel_lst/train_annots/train.lst

1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	barer_10.jpg
2	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	ht_16.jpg
3	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	extensionservicces_1 (13).jpg
4	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0

In [72]:
gt_df[sel_cols].tail(nvalid).to_csv(
    "prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst

18	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	extensionservicces_1 (11).jpg
19	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	extensionservicces_1 (8).jpg
20	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	extensionservicces_1 (7).jpg
21	0	0	0	0	1	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	

In [74]:
train_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/train_annots/train.lst", sep="\t", header=None
)
images = list(train_df[train_df.columns[-1]].values)
for image in images:
    shutil.copy(
        "prepare_data/img_clf_multilabel_lst/all_images_gt/" + image,
        "prepare_data/img_clf_multilabel_lst/train_imgs/",
    )

In [75]:
valid_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst", sep="\t", header=None
)
print(valid_df)
images = list(valid_df[valid_df.columns[-1]].values)
for image in images:
    # print image
    print(image)
    shutil.copy(
        "prepare_data/img_clf_multilabel_lst/all_images_gt/" + image,
        "prepare_data/img_clf_multilabel_lst/valid_imgs/",
    )

   0    1    2    3    4    5    6    7    8    9    ...  123  124  125  126  \
0   18    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
1   19    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
2   20    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
3   21    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
4   22    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
5   23    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   
6   24    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
7   25    0    0    0    0    1    0    0    0    0  ...    0    0    0    0   

   127  128  129  130  131                            132  
0    0    0    0    0    0  extensionservicces_1 (11).jpg  
1    0    0    0    0    0   extensionservicces_1 (8).jpg  
2    0    0    0    0    0   extensionservicces_1 (7).jpg  
3    0    0    0    0    0   extensions